# Aprendizaje, Generalización y Sobreajuste
## Validación cruzada
### Grupo de Meteorología de Santander
### 22 Nov 2018

# 1 Introducción

## Modelos de aprendizaje

Para que un problema sea adecuado para su resolución mediante técnicas de aprendizaje automático la condición fundamental es la existencia de datos para alimentar (entrenar) el algoritmo de aprendizaje.

De esta forma, a partir de un conjunto de datos de entrenamiento que se considera representativo de la distribución que se quiere modelar, nuestro modelo de aprendizaje viene dado por:

* El **algoritmo usado** para aprender el patrón y 
* La familia de **funciones** 
* Los **datos** que utilicemos para aproximar el patrón a aprender. 

De este modo, el proceso de aprendizaje dependerá de estos tres elementos, los cuales establecerán nuestras limitaciones e incertidumbres a la hora tanto de aprender el patrón como de realizar predicciones con el patrón aprendido.

## Generalización y sobreajuste

Recordemos que el objetivo principal del modelo aprendido es que tenga **la capacidad de generalizar**, es decir, la capacidad de funcionar bien para nuevos datos que no forman parte de la muestra de entrenamiento (por ejemplo, una muestra de datos de test). En caso contario, diremos que el modelo está **sobreajustado a la muestra de entrenamiento**. 

**La introducción de grados de libertad en la familia de funciones consideradas en el aprendizaje suele dar lugar a modelos sobreajustados**, por lo que suele ser conveniente partir de los modelos más simples e ir introduciendo grados de libertad progresivamente si fuera necesario.



En esta práctica utilizaremos...

* modelo: Regresion lineal
* Dataset: Auto (paquete ISLR)
* Validación: error medio absoluto (MAE) y error cuadrático medio (MSE)
* Librerías de R:

In [ ]:
library(ISLR)
library(caret)

# 2 Carga y transformación de datos

El dataset Auto (paquete `ISLR`) contiene información sobre las características de 392 vehículos. Puede verse una descripción detallada del conjunto de datos mediante: `help("Auto", package = "ISLR")`

In [ ]:
data(Auto)
str(Auto)

In [ ]:
# Conversión de libras a Kg
Auto$weight <- Auto$weight * 0.453592

Comenzamos con un análisis preliminar de nuestros datos:

In [ ]:
pairs(Auto)

En este ejemplo solo nos interesa la relación entre el peso (weight) y la potencia (horsepower).


In [ ]:
pairs(weight ~ horsepower, data = Auto, col = c("orange", "green3", "blue")[unclass(Auto$origin)])

#### Supuesto:

Imaginemos que una empresa X nos proporciona únicamente los registros o datos europeos para que obtengamos un modelo capaz de estimar los pesos de los coches americanos en función de la potencia.

In [ ]:
# Ejecutar esto una sola vez
Auto$origin <- as.character(factor(Auto$origin, labels = c("American", "Japanese", "European")))

In [ ]:
# muestra de test
america <- Auto[Auto$origin == "American", ]
# muestra de train
europe <- Auto[Auto$origin == "European", ]
str(america); str(europe)

Visualicemos ambos subconjuntos:

In [ ]:
plot(america$horsepower, america$weight, 
     xlab = "horsepower", ylab = "weight",
     pch = 16, col = "red",
     xlim = range(Auto$horsepower))
points(europe$horsepower, europe$weight, pch = 16)
# La función `lm` realiza el ajuste de un modelo lineal entre ambas variables:
abline(lm(weight~horsepower, data = america), col = "red")
abline(lm(weight~horsepower, data = europe))

# 3 Definición de las funciones de error

A continuación se definen las funciones que se van a emplear para determinar el error del modelo, y que será el error medio absoluto (*mean absolute error*, $MAE$) y el error cuadrático medio (*root mean square error*, $RMSE$):

In [ ]:
# Error Functions:
mae <- function(obs, est) {
  mean(abs(obs - est))
}

rmse <- function(obs, est) {
  sqrt(mean((obs - est)^2))
}

# 4 Construcción del modelo y evaluación

## 4.1 Contrucción del modelo sin validación cruzada

La función `lm` del paquete básico `stats` realiza el ajuste del modelo lineal:

In [ ]:
reg <- lm(weight ~ horsepower, data = europe) 
summary(reg)

Se extraen los valores ajustados por el modelo ("fitted values") de `reg`, es decir, el resultado de la applicación del modelo ajustado a los propios datos de entrenamiento `yest`, para comparar la estimación (o predicción) del modelo con los datos reales (observados):

In [ ]:
yest <- reg$fitted.values

***
**Nota**: Este procedimiento es idéntico (salvando posibles errores de redondeo) a utilizar nuestro modelo para predecir la respuesta (*weight*) en función del predictor (*horsepower*), empleando exactamente el mismo subcontunto que se utilizó en el proceso de entrenamiento:

In [ ]:
yest_bis <- predict(object = reg, newdata = data.frame(horsepower = europe$horsepower))


***

In [ ]:
plot(europe$weight, type = 'l')
lines(yest, col = 'blue')
legend("topleft", c("obs","pred"), lty=1, col = c("black", "blue"))

Por último, se calcula el error de la estimación (MAE y RMSE):

In [ ]:
mae(obs = europe$weight, yest)
rmse(obs = europe$weight, yest)

Adicionalmente, la correlación proporciona otra medida del error, relacionada con la *asociación* entre las series observada y predicha:

In [ ]:
cor(europe$weight, yest, method = "spearman")

***
El ejercicio anterior utiliza todos los datos para entrenar y validar el modelo. Es decir, la validación se realiza utilizando como conjunto de validación el mismo conjunto utilizado como "train" (conjunto de entrenamiento). Por lo tanto no podemos estimar la capacidad de generalización o sobreajuste del modelo. 

Si la validación se realiza sobre un conjunto independiente de la muestra de entrenamiento el error esperable es mayor. Ilustraremos esto con los siguientes ejemplos:
***

## 4.2 Construcción del modelo y validación *hold-out* 

El modo más básico de analizar el error de mi modelo es dividiendo la muestra en subconjuntos disjuntos (hold out). En este primer caso consideraremos sólo dos conjuntos, uno de train y otro de validación con la mitad de datos (instancias, i.e. filas) en cada uno. 

In [ ]:
n <- nrow(europe)
ind <- order(europe$horsepower)[1:ceiling(n/2)]
europe.train <- europe[ind, ]
europe.val <- europe[-ind,]

plot(america$horsepower, america$weight, 
     xlab = "horsepower", ylab = "weight",
     pch = 16, col = "red",
     xlim = range(Auto$horsepower))
points(europe.train$horsepower, europe.train$weight, pch = 16, col = "black")
points(europe.val$horsepower, europe.val$weight, pch = 21, bg = "white", cex = 0.8)

Aplicamos de nuevo las funciones `lm` (para ajustar el modelo) y  `predict` (para aplicar el modelo sobre los datos de "train" y de "validación") y calculamos el RMSE:

In [ ]:
reg0 <- lm(weight~horsepower, data = europe.train)
yest0.train <- predict(reg0, newdata = data.frame(horsepower = europe.train$horsepower))
yest0.val <- predict(reg0, newdata = data.frame(horsepower = europe.val$horsepower))
rmse(europe.train$weight, yest0.train)
rmse(europe.val$weight, yest0.val)

***
Como resulta obvio, Hemos entrenado con los valores de horsepower menores, por lo que **nuestra muestra no es representativa de la población**. 
***

### 4.2.1 Muestreo aleatiorio para obtener las muestras de train y de validación

Para intentar minimizar el sesgo de nuestra muestra de entrenamiento, una solución es aleatorizar la selección. En este ejemplo utilizamos la función `sample` para obtener un índice de registros aleatoria. 

In [ ]:
n <- nrow(europe)
set.seed(1)
ind <- sample(1:n, ceiling(n/2))
europe.train <- europe[ind, ]
europe.val <- europe[-ind,]

plot(america$horsepower, america$weight, 
     xlab = "horsepower", ylab = "weight",
     pch = 16, col = "red",
     xlim = range(Auto$horsepower))
points(europe.train$horsepower, europe.train$weight, pch = 16, col = "black")
points(europe.val$horsepower, europe.val$weight, pch = 21, bg = "white", cex = 0.8)

Aplicamos de nuevo la funciones `lm` (para ajustar el modelo) y  `predict` (para aplicar el modelo sobre los datos de "train" y de "validación") y calculamos el RMSE:

In [ ]:
reg1 <- lm(weight~horsepower, data = europe.train)
yest1.train <- predict(reg1, newdata = data.frame(horsepower = europe.train$horsepower))
yest1.val <- predict(reg1, newdata = data.frame(horsepower = europe.val$horsepower))
rmse(europe.train$weight, yest1.train)
rmse(europe.val$weight, yest1.val)

***
Al considerar una muestra representativa de la variabilidad de la población el error de validación es más bajo y se asemeja más al error de train que en el ejemplo anterior.

**El modelo entrenado con una muestra aleatoria de `europe` tiene más capacidad de generalización** ya que:

Decimos que existe sobreajuste cuando el error de train y el de validación son muy diferentes. Un modelo con capacidad de generalización, no sobreajustado, es aquel para el que ambos errores, en la muestra de entrenamiento y de test, son similares/comparables.

***

A pesar de ello, esta metodología tiene dos inconvenientes potenciales:

 1.- La estimación del error en el conjunto de validación puede variar mucho en función de la partición considerada:

In [ ]:
plot(europe$horsepower, europe$weight, pch = "*")
for (i in c(1:5)) {
  ind <- sample(1:n, ceiling(n/2))
  europe.train.i <- europe[ind, ]
  europe.val.i <- europe[-ind, ]
  reg.i <- lm(weight~horsepower, data = europe.train.i)
  yest.val.i <- predict(reg.i, data.frame(horsepower = europe.val.i$horsepower))
  abline(reg.i, lty = 2, col = "grey30")
  message("RMSE fold ", i, "= ",rmse(europe.val.i$weight, yest.val.i))
}

  2.- El error de validación puede ser sobreestimado. Dado que el modelo se entrena en un subconjunto de la muestra y que los métodos estadísticos suelen comportarse peor cuando son entrenados con pocos datos (n/2 = 40 en los ejemplos anteriores), esto puede dar lugar a una sobrestimación del error de validación respecto al obtenido considerando toda la muestra.

La validación cruzada (cross-validación) que aplicaremos un poco más adelante considera estos dos problemas. 

## 4.3 Modelización y cross-validación leave-one-out

Como alternativa al muestreo aleatorio existe el método de validación cruzada denominado leave-one-out:

* La selección de la muestra de entrenamiento NO se hace aleatoriamente, eliminando la variabilidad del error de validación.

* La muestra de entrenamiento es la mayor posible que considera una muestra de validación.

* Si un conjunto de datos tiene `N` registros, el ajuste del modelo se realiza con `N - 1` registros y el registro no considerado en el conjunto de entrenamiento se utiliza como validación o muestra independiente para validar el modelo.

* Esta operación se repite `N` veces, así todos los registros del dataset se utilizan como dato de test para un modelo entrenado con el resto de registros.


In [ ]:
ind <- 1:n
yest.2 <- numeric(length = length(ind))

for (i in ind) {
  Reg.i <- lm(weight~horsepower, data = europe, subset = ind[-i])
  yest.2[i] <- predict(Reg.i, data.frame(horsepower = europe$horsepower[i]))
}
rmse(europe$weight, yest.2)

### 4.3.1 Ejemplo anterior con la librería caret

In [ ]:
ctrl <- trainControl(method = "LOOCV")
mod <- train(weight ~ horsepower,
               data = europe,
               method = "lm",
               trControl = ctrl)
mod

In [ ]:
mod$results$RMSE

## 4.4 Modelización y cross-validación k-fold

Si el tamaño muestral es grande el método leave-one-out es computacionalmente costoso. Para evitar este coste surge otro método de validación cruzada: **El método k-fold** en el que se hace un leave-one-out por "bloques" o "folds": 

* se divide la muestra en `k` subconjuntos.

* Se ajustan `k` modelos, considerando en cada caso un bloque como conjunto de validación y los `k-1` restantes como muestra de entrenamiento. 

* La estimación dependerá de como se realice la partición de los datos. La variabilidad mayor que en el caso del leave-one-out.

* Con un número suficiente de subconjuntos, se obtienen los mismos resultados y conclusiones que las obtenidas con un leave-one-out. 

Consideramos el ejemplo anterior con 10 subconjuntos y vamos paso a paso.

1) Dividimos la muestra en 10 subconjuntos (`k = 10`)

In [ ]:
k <- 10
# Número de registros (instancias u observaciones) en nuestro dataset.
n <- nrow(europe) 
# factor de números aleatorios con k levels (del 1 al 10) 
set.seed(1)
split.factor <- sample(rep(1:10, each = ceiling(n/k)), n) 
# Lista que en cada "slot" contiene un fold 
spl.europe <- split(europe, f = split.factor)
str(spl.europe)

2) ajustamos el modelo con la función `lm` con `k-1` y predecimos (función `predict`) sobre el fold restante. Se repite la operación `k` veces (en un bucle lapply):

In [ ]:
yest.3 <- lapply(1:k, function(x) {
  reg.3 <- lm(weight~horsepower, data = do.call("rbind", spl.europe[-x]))
  predict(reg.3, data.frame(horsepower = spl.europe[[x]]$horsepower))
})
str(yest.3)
# Hacemos un plot de los valores de weight rales frente a los estimados 
plot(do.call("rbind", spl.europe)$weight, typ = "l")
lines(do.call("c", yest.3), col = "red")

3) Calculamos el error

In [ ]:
# de cada fold
rmse.val.3.folds <- lapply(1:length(spl.europe), function(x) rmse(spl.europe[[x]]$weight, yest.3[[x]]))
# de la muestra entera
rmse.val.3 <- rmse(do.call("rbind", spl.europe)$weight, do.call("c", yest.3))
plot(do.call("c", rmse.val.3.folds), ylab = "rmse", xlab = "fold", pch = "x", ylim = c(0, 400), col = "red")
abline(h = rmse.val.3, col = "red")
abline(h = rmse(obs = europe$weight, yest))

# Práctica 1

¿Es el modelo aprendido con los registros europeos adecuado para estimar el peso de los coches americanos? (¿qué ocurre cuando aplicamos el modelo a la muestra de test (objeto `america`)?)

Utiliza las funciones de visualización (`plot`, `points`, `lines`, `abline`, ...) para ilustrar los resultados.

Escribe el código a continuación:

In [ ]:
reg.3.all <- lm(weight~horsepower, data = europe)
#(...)

plot(do.call("c", rmse.val.3.folds), ylab = "rmse", xlab = "fold", pch = "x", ylim = c(0, 400))# error de validación obtenido anteriormente para cada fold
abline(h = rmse.val.3)# error de validación global (de todos los folds) obtenido anteriormente 

# abline(...

#(...)

# Práctica 2

Utiliza la librería caret para reproducir el último ejemplo de cross-validación con k-fold y obtén el error de validación global.

Escribe el código a continuación:

# Práctica 3

Utiliza los datos de los coches americanos para estimar los pesos de los coches europeos por un lado y la de los japoneses por otro.

¿Es el modelo aprendido adecuado para estimar el peso de los coches europeos y japoneses?

Utiliza caret y las funciones de visualización (`plot`, `points`, `lines`, `abline`, ...) para ilustrar los resultados. 

Escribe el código a continuación: